In [ ]:
# =============================================================================
# Project: AI Palmistry Analysis (AI 베테랑 역술가)
# Version: V17.5 (Refactored)
# Description: YOLOv8-Pose 기반 손금 탐지 및 관상학적 해석 시스템
# Author: [Your Name/GitHub ID]
# =============================================================================

import os
import math
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Any, Union

# External Libraries
try:
    from ultralytics import YOLO
    import mediapipe as mp
    from PIL import ImageFont, ImageDraw, Image
    from IPython.display import display, HTML
    from google.colab import drive, files
except ImportError as e:
    print(f"❌ 필수 라이브러리가 설치되지 않았습니다: {e}")
    print("👉 !pip install ultralytics mediapipe")

# =============================================================================
# [Configuration] 설정 및 데이터 관리
# =============================================================================
class PalmConfig:
    """분석에 필요한 상수, 임계값, 텍스트 데이터를 관리하는 설정 클래스"""

    # 폰트 경로 (Colab 환경 기준)
    FONT_PATH = '/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf'

    # 분석 임계값
    THRESHOLDS = {
        'gap': [0.01, 0.03, 0.05, 0.07, 0.10, 0.13, 0.18],
        'life_len': [0.30, 0.38, 0.44, 0.50, 0.56, 0.62, 0.70],
        'curv': [1.02, 1.05, 1.09, 1.14, 1.20],
        'slope': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8]
    }

    # 시각화 색상 (BGR Format) - 파스텔톤
    COLORS = {
        0: {'ko': '운명선', 'name': 'fate', 'color': (140, 150, 255)}, # Pink
        1: {'ko': '두뇌선', 'name': 'head', 'color': (255, 230, 150)}, # Sky Blue
        2: {'ko': '감정선', 'name': 'heart', 'color': (255, 120, 120)}, # Light Red
        3: {'ko': '생명선', 'name': 'life', 'color': (120, 255, 230)}, # Mint
    }

    # 해석 텍스트 데이터베이스
    TEXTS = {
        'intro': [
            "어서 오십시오. 손바닥을 보니, 남들에게는 말 못 할 고민과 당신만의 뜨거운 열망이 동시에 느껴지는군요.",
            "손금은 단순한 주름이 아닙니다. 당신이 살아온 치열한 흔적이자, 앞으로 펼쳐질 운명의 설계도입니다.",
            "사람의 얼굴이 다 다르듯, 손금 또한 우주에서 유일무이합니다. 지금 당신의 손에는 위기 속에서도 기회를 찾아내는 비범한 잠재력이 숨어 있습니다."
        ],
        'life_start': {
            'glue': "🔒 <b>[신중한 완벽주의자]</b><br>돌다리도 두드려 보고 건너는 신중함을 타고났습니다.",
            'tight': "👀 <b>[배려심 깊은 평화주의자]</b><br>남의 눈치를 조금 보는 편입니다.",
            'normal_tight': "🛡️ <b>[외유내강형]</b><br>안전지향적이지만 확신이 서면 과감하게 밀고 나가는 스타일입니다.",
            'normal': "⚖️ <b>[처세술의 달인]</b><br>가장 이상적인 밸런스입니다!",
            'normal_loose': "🏃 <b>[자수성가의 싹]</b><br>부모님 품을 일찍 떠나거나 스스로의 힘으로 서려는 독립심이 강합니다.",
            'loose': "🦅 <b>[자유로운 영혼]</b><br>남들이 '예'라고 할 때 '아니오'라고 말할 수 있는 용기가 있습니다.",
            'free': "🚀 <b>[시대를 앞서가는 혁신가]</b><br>와우, 완전히 분리된 KY선! 스티브 잡스 같은 혁신가들의 손금입니다."
        },
        'life_length_book': {
            'long': "🧬 <b>[축복받은 장수운]</b><br>생명선이 손목까지 시원하게 뻗었습니다. 타고난 면역력이 강합니다.",
            'middle': "🏃 <b>[균형 잡힌 웰빙 라이프]</b><br>생명선이 손목 위쪽에서 깔끔하게 마무리됩니다.",
            'short': "⚠️ <b>[짧고 굵은 불꽃]</b><br>수명이 짧다기보다 에너지를 한 번에 몰아서 쓰는 '단기 집중형'입니다."
        },
        'life_curve_book': {
            'big': "💪 <b>[지치지 않는 에너자이저]</b><br>금성구가 매우 발달하여 가만히 있으면 병이 나는 스타일입니다.",
            'moderate': "🌿 <b>[외유내강의 정석]</b><br>겉으로는 온화해 보이지만 내면에는 단단한 심지가 있습니다.",
            'small': "🍃 <b>[섬세한 감수성]</b><br>타고난 체력이 강한 편은 아니기에 쉽게 피로를 느낄 수 있습니다."
        },
        'life_direction_book': {
            'thumb': "🏠 <b>[가정이 1순위]</b><br>생명선 끝이 엄지 쪽으로 감깁니다. 가족과 안정이 최우선입니다.",
            'wrist': "🌲 <b>[안정을 택하는 나무]</b><br>모험보다는 확실한 안정을 선호합니다.",
            'pinky': "✈️ <b>[세계를 누비는 역마살]</b><br>생명선 끝이 바깥으로 뻗어 나갑니다. 변화가 곧 기회입니다."
        },
        'head_length_book': {
            'long': "🐢 <b>[깊이 생각하는 철학자]</b><br>생각의 꼬리가 깁니다. 돌발 상황에서도 전략적으로 움직입니다.",
            'short': "⚡ <b>[직관이 번뜩이는 승부사]</b><br>\"이거다!\" 싶으면 앞뒤 재지 않고 바로 실행에 옮깁니다."
        },
        'head_shape_book': {
            'straight': "📐 <b>[팩트폭격기 이과형]</b><br>감정 호소보다는 논리와 데이터, 팩트를 중요시합니다.",
            'curved': "🎨 <b>[감성 충만 문과형]</b><br>딱딱한 논리보다 사람의 마음을 먼저 읽는 능력을 가졌습니다."
        },
        'head_destination_book': {
            'mars_2': "💰 <b>[돈 냄새를 맡는 실리파]</b><br>현실 감각이 매우 뛰어납니다.",
            'moon_upper': "👑 <b>[사람을 이끄는 리더]</b><br>현실적인 문제 해결 능력과 창의적인 비전을 동시에 갖췄습니다.",
            'moon_middle': "🎉 <b>[아이디어 뱅크]</b><br>당신의 머릿속에는 남들을 즐겁게 할 아이디어가 가득하군요!",
            'moon_lower': "📚 <b>[영혼이 맑은 예술가]</b><br>물질적인 성공보다는 정신적인 가치와 마음의 평화를 추구합니다.",
            'mercury': "🕵️ <b>[비상한 전략가]</b><br>눈치가 100단이고 임기응변에 능합니다.",
            'sun': "💎 <b>[화려한 스타성]</b><br>자신을 돋보이게 하는 방법을 본능적으로 알고 있습니다.",
            'saturn': "🙏 <b>[끝없는 구도자]</b><br>현실의 만족보다 더 높은 가치를 좇습니다."
        },
        'heart_start_book': {
            'standard': [
                "⚖️ <b>[이성과 감정의 황금비율]</b><br>어떤 상황에서도 평정심을 유지할 줄 아는군요.",
                "⚖️ <b>[믿음직한 파트너]</b><br>당신은 감정에 휘둘리지 않고 중심을 잡을 줄 압니다."
            ],
            'high': [
                "🔥 <b>[브레이크 없는 열정: 금사빠]</b><br>사랑에 빠지면 물불 가리지 않는 열정파입니다!",
                "💖 <b>[감수성 풍부한 로맨티시스트]</b><br>작은 이벤트에도 큰 감동을 받는 낭만파입니다.",
                "✨ <b>[재지 않는 솔직함]</b><br>내 마음 가는 대로 직진하는 솔직함이 매력입니다."
            ],
            'low': [
                "❄️ <b>[침착한 포커페이스]</b><br>겉으로 감정을 잘 드러내지 않습니다.",
                "🧊 <b>[냉철한 이성주의자]</b><br>감정보다는 현실적인 조건을 먼저 고려합니다."
            ]
        },
        'heart_shape_book': {
            'straight': "📏 <b>[돌직구 직진남녀]</b><br>밀당? 그런 건 딱 질색이시죠.",
            'curved': "🌊 <b>[따뜻한 감성 힐러]</b><br>타인의 감정을 내 것처럼 느끼는 공감 능력을 가졌습니다.",
            'hybrid': "🎭 <b>[알다가도 모를 반전 매력]</b><br>냉정과 열정 사이를 오가는 양면성이 있습니다."
        },
        'heart_end_book': {
            'cross': "🔒 <b>[집착과 소유의 끝판왕]</b><br>당신에게 사랑은 '전부 아니면 전무'입니다.",
            'jupiter': "🏰 <b>[신중한 로맨티시스트]</b><br>존경할 수 있는 대상을 찾아 천천히 사랑을 키워갑니다.",
            'index_border': "🌻 <b>[해바라기]</b><br>사랑 앞에서는 맹목적인 바보가 됩니다.",
            'index_middle_down': "❤️‍🔥 <b>[가슴 속 마그마]</b><br>겉보기엔 얌전해 보이지만 가슴 속에는 뜨거운 열정이 있습니다.",
            'index_middle_end': "💍 <b>[결혼하고 싶은 사람 1위]</b><br>성실하고 가정적입니다.",
            'middle_border': "🍲 <b>[금방 끓고 식는 냄비]</b><br>열정이 확 타올랐다가 금방 시들해지는 경향이 있습니다.",
            'middle_down': "😎 <b>[쿨내 진동하는 자유인]</b><br>구속받는 것을 싫어하고 프라이버시를 중요하게 생각합니다.",
            'index_down_life': "🍎 <b>[위험한 유혹]</b><br>장애물이 있는 '금단의 사랑'에 끌리는 경향이 있습니다.",
            'middle_down_life': "👑 <b>[내 멋대로 하는 사랑]</b><br>자기중심적인 성향이 강해 내 뜻대로 상대가 움직여주길 바랍니다."
        },
        'fate_path': {
            'support': "🤝 <b>[최고의 참모] (옅은 운명선)</b><br>누군가를 보좌할 때 빛을 발하는 서포터형 인재입니다.",
            'self_made': "🌟 <b>[맨주먹 성공 신화]</b><br>오직 자신의 실력과 노력으로 자수성가할 운명입니다.",
            'popular': "💖 <b>[대중의 별, 인기운]</b><br>'인복'과 '인기'를 타고났습니다.",
            'late': "🐢 <b>[대기만성의 아이콘]</b><br>초년의 고생은 훗날의 영광을 위한 밑거름입니다."
        },
        'special_signs': {
            'm_sign': "🏆 <b>[전설의 대박 징조, M자 손금!]</b><br>축하합니다! 부와 명예를 거머쥘 수 있는 희귀한 길상입니다.",
            'rich_triangle': "💰 <b>[돈이 마르지 않는 삼각 창고]</b><br>재물 창고가 튼튼하여 평생 돈 걱정 없이 살 상입니다.",
            'no_fate': "🦅 <b>[길 없는 곳이 내 길] (무운명선)</b><br>정해진 운명의 틀이 없기에 당신이 걷는 곳이 곧 길이 됩니다."
        },
        'fate_start_book': {
            'venus': ["👪 <b>[가문의 영광]</b><br>가족의 전폭적인 지원을 받아 운이 열립니다.", "👪 <b>[든든한 울타리]</b><br>상속운이 있어 풍요로운 삶을 누릴 가능성이 높습니다."],
            'mars_1': ["🦁 <b>[난세의 영웅]</b><br>사회가 혼란스러울 때 더 큰 힘을 발휘합니다.", "🦁 <b>[불굴의 투사]</b><br>역경을 먹고 자라는 나무처럼 강인합니다."],
            'venus_moon_middle': ["🛠️ <b>[성실함의 승리]</b><br>요행을 바라지 않고 오직 땀과 노력으로 운을 개척합니다.", "🛠️ <b>[믿음직한 개척자]</b><br>스스로 만드는 길을 즐기는 타입입니다."],
            'mars_2': ["🤝 <b>[인복의 제왕]</b><br>성공의 열쇠는 파트너에게 있습니다.", "🤝 <b>[귀인의 도움]</b><br>어려운 순간마다 기적처럼 돕는 손길이 있습니다."],
            'moon': ["🌟 <b>[대중의 아이돌]</b><br>타고난 스타성으로 사람들을 끌어당깁니다.", "🌟 <b>[매력 자본가]</b><br>실력보다는 매력으로 승부할 때 더 큰 성과를 냅니다."],
            'mars_plain': ["🔥 <b>[진흙 속의 연꽃]</b><br>초년 고생 끝에 중년 이후 잠재력이 폭발합니다.", "🔥 <b>[인내의 화신]</b><br>대기만성의 주인공은 바로 당신입니다."],
            'above_life': ["🐢 <b>[노력은 배신하지 않는다]</b><br>피나는 노력 끝에 해당 분야의 일인자가 됩니다.", "🐢 <b>[성실함의 결정체]</b><br>티끌 모아 태산을 이루는 운명입니다."],
            'above_head': ["💡 <b>[천재적 두뇌]</b><br>타고난 재능과 아이디어로 승부합니다.", "💡 <b>[늦깎이 천재]</b><br>머리를 쓰는 일에서 크게 성공합니다."],
            'above_heart': ["🏆 <b>[인생은 50부터]</b><br>말년에 화려한 인생 역전을 이룹니다.", "🏆 <b>[노익장 과시]</b><br>끝이 좋은 사람입니다."],
            'wrist_straight': ["🚀 <b>[직진 본능]</b><br>타협을 모르는 강직함으로 한 길만 팝니다.", "🚀 <b>[천상천하 유아독존]</b><br>독보적인 존재감을 드러냅니다."]
        },
        'fate_end_book': {
            'saturn': ["🏅 <b>[고독한 승부사]</b><br>재물보다는 명예와 성취감을 중요시합니다.", "🏅 <b>[자수성가의 끝판왕]</b><br>오직 자신의 힘으로 정상에 오릅니다."],
            'jupiter': ["🤝 <b>[야망의 정치가]</b><br>권력과 지위를 얻어야 직성이 풀립니다.", "🤝 <b>[신뢰의 아이콘]</b><br>돈보다는 사람을 남기는 장사를 합니다."],
            'sun': ["💰 <b>[화려한 재력가]</b><br>타고난 연예인! 화려한 삶을 살게 됩니다.", "💰 <b>[행운의 총아]</b><br>즐기면서 돈까지 버는 행운아입니다."],
            'head_stop': ["🛑 <b>[신중한 현실주의자]</b><br>30대 중반, 큰 결단이 필요합니다.", "🛑 <b>[지략가의 딜레마]</b><br>성공은 당신의 판단력에 달려 있습니다."],
            'heart_stop': ["❤️ <b>[열정의 배신]</b><br>일에 몰두하느라 사랑을 놓치지 마세요.", "❤️ <b>[은퇴 후의 여유]</b><br>말년에는 마음 편한 안분지족을 꿈꿉니다."]
        },
        'spicy_title': "🔞 <b>[Secret] 베테랑이 본 당신의 '밤의 얼굴'</b>",
        'libido': {'level_5': "🔥 <b>[폭주하는 정복자]</b><br>밤이 되면 굶주린 포식자로 돌변합니다.", 'level_3': "🍷 <b>[농밀한 테크니션]</b><br>섬세한 터치로 상대를 황홀경으로 이끕니다.", 'level_1': "🍃 <b>[플라토닉]</b><br>육체적 관계보다 정신적 교감을 중요시합니다."},
        'seduction': {'bad_charmer': "😈 <b>[영혼의 포식자]</b><br>상대를 조련하는 치명적인 매력이 있습니다.", 'hot_lover': "❤️‍🔥 <b>[본능의 노예]</b><br>필 꽂히면 앞뒤 안 보고 직진합니다.", 'shy_fox': "🦊 <b>[낮져밤이]</b><br>순진해 보이지만 밤에는 180도 돌변합니다.", 'cool_friend': "😎 <b>[쿨한 관계]</b><br>질척이는 건 딱 질색입니다.", 'devoted_puppy': "🐶 <b>[자발적 노예]</b><br>상대의 가학적인 명령에도 기꺼이 순응합니다."},
        'fantasy': {'deep': "🦄 <b>[판타지 마니아]</b><br>머릿속에 야릇한 시나리오가 가득합니다.", 'real': "💼 <b>[현실주의]</b><br>상상보다는 눈앞의 확실한 자극을 선호합니다."},
        'outro': ["운명은 정해진 것이 아니라, 당신의 의지가 만들어가는 것입니다.", "오늘이 바로 그 시작점입니다. 당신의 가능성을 믿으세요!"]
    }

# =============================================================================
# [Logic] AI Palm Interpreter Engine (V16.2 Logic)
# =============================================================================
class PalmInterpreter:
    def __init__(self):
        self.config = PalmConfig()
        self.texts = self.config.TEXTS
        self.TH = self.config.THRESHOLDS

    def get_step(self, value, thresholds):
        for i, th in enumerate(thresholds):
            if value < th: return i
        return len(thresholds)

    # --- Analysis Logic ---
    def analyze_life_length(self, f, h, wy):
        pts = f['points']; end_y = max(pts, key=lambda p: p[1])[1]
        ql = wy - (h * 0.25)
        return 'long' if end_y > ql + (h * 0.1) else 'middle' if end_y > ql else 'short'

    def analyze_life_curve(self, f, m):
        pts = f['points']
        if pts[0][0] < pts[-1][0]:
             ax = max(pts, key=lambda p: p[0])[0]
             if ax > m['토성'][0]: return 'big'
             elif ax > m['목성'][0]: return 'moderate'
             else: return 'small'
        return 'moderate'

    def analyze_life_dir(self, f, m):
        pts = f['points'];
        if len(pts) < 5: return 'wrist'
        dx = pts[-1][0] - pts[-5][0]
        if dx < -5: return 'thumb'
        elif dx > 5: return 'pinky'
        else: return 'wrist'

    def analyze_head_len(self, f, m):
        rx = m['태양'][0]; pts = f['points']
        ex = pts[-1][0] if pts[0][0] < pts[-1][0] else pts[0][0]
        return 'long' if ex > rx else 'short'

    def analyze_head_shape(self, f):
        return 'straight' if f['curv'] < 1.06 else 'curved'

    def analyze_head_dest(self, f, m, h):
        pts = f['points']
        ep = pts[-1] if pts[0][0] < pts[-1][0] else pts[0]
        sp = pts[0] if pts[0][0] < pts[-1][0] else pts[-1]
        if ep[1] < sp[1] - (h * 0.1):
            if ep[0] > m['수성'][0]: return 'mercury'
            elif ep[0] > m['태양'][0]: return 'sun'
            else: return 'saturn'
        else:
            m2y = m['제2화성'][1]; wy = m['地'][1]
            if ep[1] < m2y + (h * 0.1): return 'mars_2'
            mt = m2y + (h * 0.1); mr = wy - mt
            if ep[1] < mt + (mr * 0.33): return 'moon_upper'
            elif ep[1] < mt + (mr * 0.66): return 'moon_middle'
            else: return 'moon_lower'

    def analyze_heart_start_book(self, f, h, py):
        pts = f['points']; sp = pts[0] if pts[0][0] > pts[-1][0] else pts[-1]
        sy = sp[1]; ph = h * 0.6; ql = py + (ph * 0.25)
        if sy < ql - (ph * 0.08): return 'high'
        elif sy > ql + (ph * 0.05): return 'low'
        else: return 'standard'

    def analyze_heart_shape_book(self, f):
        return 'straight' if f['curv'] < 1.04 else 'curved' if f['curv'] > 1.10 else 'hybrid'

    def analyze_heart_end_book(self, f, m):
        pts = f['points']; ep = pts[0] if pts[0][0] < pts[-1][0] else pts[-1]
        jx = m['목성'][0]; sx = m['토성'][0]; lsy = m['火'][1]
        if ep[0] < jx - 30: return 'cross'
        if ep[1] > lsy: return 'index_down_life' if abs(ep[0]-jx) < abs(ep[0]-sx) else 'middle_down_life'
        else:
            if abs(ep[0]-jx) < 20: return 'jupiter'
            elif abs(ep[0]-sx) < 20: return 'middle_down'
            if jx < ep[0] < sx: return 'index_middle_end' if ep[1] < m['목성'][1] + 20 else 'index_middle_down'
            return 'middle_border' if ep[0] > sx else 'index_border'

    def analyze_fate_start_book(self, f, m, features):
        pts = f['points']; sp = pts[0] if pts[0][1] > pts[-1][1] else pts[-1]
        sx = sp[0]; sy = sp[1]
        if sy > m['地'][1] * 0.95: return 'wrist_straight'
        if 'head' in features and sy < features['head']['points'][0][1]: return 'above_head'
        if 'heart' in features and sy < features['heart']['points'][0][1]: return 'above_heart'
        center_x = m['火'][0]; width = abs(m['목성'][0] - m['수성'][0])
        if sx < center_x - (width * 0.2): return 'venus'
        elif sx > center_x + (width * 0.2): return 'moon'
        elif sx < center_x - (width * 0.1): return 'mars_1'
        elif sx > center_x + (width * 0.1): return 'mars_2'
        return 'venus_moon_middle' if sy > m['地'][1] * 0.8 else 'mars_plain'

    def analyze_fate_end_book(self, f, m, features):
        pts = f['points']; ep = pts[0] if pts[0][1] < pts[-1][1] else pts[-1]
        if 'head' in features and abs(ep[1] - features['head']['points'][len(features['head']['points'])//2][1]) < 30: return 'head_stop'
        if 'heart' in features and abs(ep[1] - features['heart']['points'][len(features['heart']['points'])//2][1]) < 30: return 'heart_stop'
        ex = ep[0]
        if ex < m['목성'][0] + 20: return 'jupiter'
        elif ex > m['태양'][0] - 20: return 'sun'
        else: return 'saturn'

    def interpret(self, features, mounts, hand_metrics):
        html = """<div class="palm-wrapper" style="font-family:'Malgun Gothic',sans-serif; background:#f0f2f5; padding:30px; border-radius:30px;">
        <div style="background:#fff; padding:15px 25px; border-radius:20px; display:inline-block; border-left:8px solid #e74c3c; margin-bottom:20px;">
        <h2 style="margin:0; font-weight:900; color:#1a202c;">🏮 AI 베테랑 역술가 리포트</h2></div><div style="display:flex; overflow-x:auto; gap:20px; padding-bottom:10px;">"""

        def card(title, color, text):
            return f"""<div style="flex:0 0 360px; background:#fff; border-radius:25px; padding:30px; border-top:12px solid {color}; box-shadow:0 15px 35px rgba(0,0,0,0.1);">
            <div style="font-size:1.4em; font-weight:900; margin-bottom:20px; color:#1a202c;">{title}</div><div style="font-size:1.05em; line-height:1.8; color:#2d3748;">{text}</div></div>"""

        # 1. 생명선
        if 'life' in features:
            f = features['life']; msg = []
            gap = features.get('head_life_gap', 0)
            step = self.get_step(gap, self.TH['gap'])
            keys = ['glue', 'tight', 'normal_tight', 'normal', 'normal_loose', 'loose', 'free', 'free']
            msg.append(self.texts['life_start'].get(keys[step], self.texts['life_start']['normal']))
            msg.append("<br>" + self.texts['life_length_book'][self.analyze_life_length(f, hand_metrics['height'], mounts['地'][1])])
            curv_res = self.analyze_life_curve(f, mounts) # For 19+
            html += card("🌿 생명선 (운명과 건강)", "#78ffe6", "".join(msg))

        # 2. 두뇌선
        if 'head' in features:
            f = features['head']; msg = []
            h_len = self.analyze_head_len(f, mounts)
            msg.append(self.texts['head_length_book'][h_len])
            msg.append("<br>" + self.texts['head_shape_book'][self.analyze_head_shape(f)])
            html += card("🧠 두뇌선 (재능과 적성)", "#ffe696", "".join(msg))

        # 3. 감정선
        if 'heart' in features:
            f = features['heart']; msg = []
            start_res = self.analyze_heart_start_book(f, hand_metrics['height'], mounts['수성'][1])
            tmpl = self.texts['heart_start_book'][start_res]
            msg.append(random.choice(tmpl) if isinstance(tmpl, list) else tmpl)
            msg.append("<br>" + self.texts['heart_shape_book'][self.analyze_heart_shape_book(f)])
            html += card("❤️ 감정선 (사랑과 관계)", "#ff7878", "".join(msg))

        # 4. 운명선
        if 'fate' in features:
            f = features['fate']; msg = []
            if f['conf'] < 0.4: msg.append(self.texts['fate_path']['support'])
            else:
                f_start = self.analyze_fate_start_book(f, mounts, features)
                msg.append(random.choice(self.texts['fate_start_book'].get(f_start, ["🌟 <b>[자수성가]</b>"])))
                f_end = self.analyze_fate_end_book(f, mounts, features)
                msg.append("<br><br>" + random.choice(self.texts['fate_end_book'].get(f_end, ["🏅 <b>[성공]</b>"])))
            if f['conf'] > 0.35: msg.append("<br><br>" + self.texts['special_signs']['rich_triangle'])
            html += card("🌟 운명선 (성공과 재물)", "#8c96ff", "".join(msg))
        else:
            html += card("🌟 운명선", "#8c96ff", self.texts['special_signs']['no_fate'])

        # 5. 19금
        msg = []
        if 'life' in features:
            k = 'level_5' if curv_res == 'big' else 'level_3' if curv_res == 'moderate' else 'level_1'
            msg.append(self.texts['libido'][k])
        if 'heart' in features and 'head' in features:
            h_curv = features['heart']['curv']
            k = 'bad_charmer' if features['heart']['len_ratio'] < 0.4 else 'hot_lover' if h_curv > 1.12 else 'shy_fox' if h_curv > 1.05 and h_len == 'long' else 'devoted_puppy' if h_curv > 1.05 else 'cool_friend'
            msg.append("<br><br>" + self.texts['seduction'][k])
        html += card(self.texts['spicy_title'], "#e74c3c", "".join(msg))

        html += "</div></div>"
        return html

# =============================================================================
# [Visualization] 시각화 엔진 (V17.5: 라벨 디자인 완전 일치)
# =============================================================================
class PalmAnalysisVisualizer:
    def __init__(self, model_path):
        self.model = YOLO(model_path)
        self.interpreter = PalmInterpreter()
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(static_image_mode=True, max_num_hands=1)
        self.font_path = PalmConfig.FONT_PATH
        self.line_config = PalmConfig.COLORS

    def apply_clahe(self, img):
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        return cv2.cvtColor(cv2.merge((clahe.apply(l), a, b)), cv2.COLOR_LAB2BGR)

    def draw_styled_label(self, img_bgr, text, position, color_bgr):
        img_pil = Image.fromarray(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(img_pil)
        try: font = ImageFont.truetype(self.font_path, 65)
        except: font = ImageFont.load_default()

        bbox = draw.textbbox(position, text, font=font)
        padding = 20
        rect_coords = [bbox[0]-padding, bbox[1]-padding, bbox[2]+padding, bbox[3]+padding]

        # [수정] 배경 박스: 선 색상으로 채움 (BGR -> RGB)
        color_rgb = color_bgr[::-1]
        draw.rounded_rectangle(rect_coords, radius=30, fill=color_rgb)

        # [수정] 글씨 색상: 배경 밝기에 따라 자동 결정 (검정/흰색)
        brightness = (color_rgb[0]*0.299 + color_rgb[1]*0.587 + color_rgb[2]*0.114)
        text_color = (0, 0, 0) if brightness > 140 else (255, 255, 255)

        # 테두리 (흰색으로 살짝 강조)
        draw.rounded_rectangle(rect_coords, radius=30, outline=(255, 255, 255), width=3)
        draw.text(position, text, font=font, fill=text_color)

        return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

    def visualize(self, image_path):
        origin_img = cv2.imread(image_path)
        if origin_img is None: return
        h, w = origin_img.shape[:2]
        canvas_img = origin_img.copy()

        # 1. 랜드마크
        img_rgb_mp = cv2.cvtColor(origin_img, cv2.COLOR_BGR2RGB)
        mp_results = self.hands.process(img_rgb_mp)

        mounts = {}; hand_metrics = {}; p17 = np.array([0,0]); p0 = np.array([0,0])

        if mp_results.multi_hand_landmarks:
            lm = mp_results.multi_hand_landmarks[0].landmark
            def p(idx): return np.array([int(lm[idx].x * w), int(lm[idx].y * h)])
            hand_metrics['height'] = np.linalg.norm(p(12) - p(0))
            hand_metrics['palm_width'] = np.linalg.norm(p(5) - p(17))
            p17, p0 = p(17), p(0)
            edge_center = (p17 + p0) * 0.5; palm_center = p(9)
            vec = edge_center - palm_center
            mars_pos = edge_center + (vec * 0.2); mars_pos = mars_pos.astype(int)
            mounts = {'月': p0, '월구': p0 * 0.7 + p17 * 0.3, '火': mars_pos, '地': p0, '목성': p(5), '토성': p(9), '태양': p(13), '수성': p(17), '제2화성': mars_pos}

            # 구 시각화
            for text, coord in [('金', p(1)), ('木', p(5)), ('土', p(13)), ('수성', p(17)), ('제2화성', mars_pos), ('월구', mounts['월구'])]:
                overlay = canvas_img.copy()
                cv2.circle(overlay, (int(coord[0]), int(coord[1])), 60, (255, 255, 255), -1)
                cv2.addWeighted(overlay, 0.5, canvas_img, 0.5, 0, canvas_img)
                img_pil = Image.fromarray(cv2.cvtColor(canvas_img, cv2.COLOR_BGR2RGB))
                draw = ImageDraw.Draw(img_pil)
                gu_font = ImageFont.truetype(self.font_path, 70)
                tw = draw.textlength(text, font=gu_font)
                draw.text((coord[0]-tw/2, coord[1]-35), text, font=gu_font, fill=(100, 100, 100))
                canvas_img = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

        # 2. 선 그리기
        ai_input = self.apply_clahe(origin_img)
        results = self.model.predict(ai_input, conf=0.05, verbose=False)
        best_lines = {}
        if results and results[0].keypoints is not None:
            kpts_all = results[0].keypoints.xy.cpu().numpy()
            clses = results[0].boxes.cls.cpu().numpy()
            confs = results[0].boxes.conf.cpu().numpy()
            for idx, kpts in enumerate(kpts_all):
                cls_id = int(clses[idx])
                if cls_id not in self.line_config: continue
                if cls_id not in best_lines or confs[idx] > best_lines[cls_id][0]:
                    best_lines[cls_id] = (confs[idx], kpts)

        for cls_id, (conf, kpts) in best_lines.items():
            cfg = self.line_config[cls_id]
            points = np.array([ [int(x), int(y)] for x, y in kpts if x > 0 ])
            if len(points) < 2: continue

            # 선 그리기 (깔끔한 파스텔톤)
            for i in range(len(points) - 1):
                cv2.line(canvas_img, tuple(points[i]), tuple(points[i+1]), cfg['color'], 14, cv2.LINE_AA)

            # 이름표 위치 선정
            name = cfg['name']
            label_pt = points[0]
            if name == 'heart': label_pt = points[0] if np.linalg.norm(points[0]-p17) < np.linalg.norm(points[-1]-p17) else points[-1]
            elif name == 'head': label_pt = points[0] if np.linalg.norm(points[0]-p17) < np.linalg.norm(points[-1]-p17) else points[-1]
            elif name == 'life': label_pt = points[0] if np.linalg.norm(points[0]-p0) < np.linalg.norm(points[-1]-p0) else points[-1]
            elif name == 'fate': label_pt = points[0] if np.linalg.norm(points[0]-p0) < np.linalg.norm(points[-1]-p0) else points[-1]

            canvas_img = self.draw_styled_label(canvas_img, cfg['ko'], tuple(label_pt), cfg['color'])

        plt.figure(figsize=(12, 12))
        plt.imshow(cv2.cvtColor(canvas_img, cv2.COLOR_BGR2RGB))
        plt.axis('off'); plt.show()

        features = self._extract_features(best_lines, hand_metrics)
        display(HTML(self.interpreter.interpret(features, mounts, hand_metrics)))

    def _extract_features(self, best_lines, metrics):
        features = {}
        for cls_id, (conf, kpts) in best_lines.items():
            pts = np.array([pt for pt in kpts if pt[0] > 0])
            if len(pts) < 2: continue
            name = self.line_config[cls_id]['name']
            curve_len = np.sum(np.sqrt(np.sum(np.diff(pts, axis=0)**2, axis=1)))
            euclidean = np.linalg.norm(pts[-1] - pts[0])
            vec = pts[-1] - pts[0]
            slope = vec[1] / (vec[0] + 1e-6)
            features[name] = {'points': pts, 'len_ratio': curve_len / metrics['height'], 'curv': curve_len / (euclidean + 1e-6), 'conf': conf, 'slope': slope}
        if 'head' in features and 'life' in features:
            features['head_life_gap'] = np.linalg.norm(features['head']['points'][0] - features['life']['points'][0]) / metrics['palm_width']
        return features

# ---------------------------------------------------------
# 실행
# ---------------------------------------------------------
BASE_PATH = '/content/drive/MyDrive/final_pro/mobileNet'
MODEL_FILE = 'num_02.pt'
model_path = os.path.join(BASE_PATH, MODEL_FILE)
if os.path.exists(model_path):
    print("👇 분석할 손 사진을 선택하세요.")
    uploaded = files.upload()
    if uploaded:
        vis = PalmAnalysisVisualizer(model_path)
        vis.visualize(list(uploaded.keys())[0])